In [1]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('sqlite://', echo=False)

# Getting labelled TESS Objects of Interest dataframe from Caltech:
toi = pd.read_csv('https://exofop.ipac.caltech.edu/tess/' + 
                  'download_toi.php?sort=toi&output=csv')

# Isolating columns we want:
toi = toi[['TIC ID',
           'TOI',
           'Epoch (BJD)',
           'Period (days)',
           'Duration (hours)',
           'Depth (mmag)',
           'Planet Radius (R_Earth)',
           'Planet Insolation (Earth Flux)',
           'Planet Equil Temp (K)',
           'Planet SNR',
           'Stellar Distance (pc)',
           'Stellar log(g) (cm/s^2)',
           'Stellar Radius (R_Sun)',
           'TFOPWG Disposition',
          ]]

toi.columns = toi.columns.str.replace(' ', '_')

toi.to_sql('toi', con=engine)

engine.execute("SELECT * FROM toi").fetchall()

[(0, 231663901, 101.01, 2458326.008874, 1.430401, 1.6387650000000002, 22.436284, 13.145007999999999, 1193.379561, 1499.044062, 0.887868, 464.73199999999997, 4.34437, 1.13487, 'KP'),
 (1, 149603524, 102.01, 2458326.0786450002, 4.411943, 3.776425, 15.285609, 15.397243, 708.767112, 1315.96894, 474.38095, 171.78799999999998, 4.73, 1.15229, 'KP'),
 (2, 336732616, 103.01, 2458327.25256, 3.547854, 3.494333, 11.377523, 14.581841, 1212.0043759999999, 1504.858953, 0.800299, 316.7399, 4.34015, 1.12109, 'KP'),
 (3, 231670397, 104.01, 2458327.6734619997, 4.087445, 5.59014, 3.900662, 12.652102000000001, 1672.217546, 1630.958624, 0.808662, 363.3885, 3.6296, 2.6644200000000002, 'KP'),
 (4, 144065872, 105.01, 2458326.50599, 2.184659, 2.869426, 12.928189999999999, 12.514586, 985.883704, 1429.144596, 0.886027, 140.7953, 4.22046, 1.30241, 'KP'),
 (5, 38846515, 106.01, 2458326.745077, 2.84938, 3.776111, 8.121677, 9.06138, 1311.3153730000001, 1534.7814369999999, 278.0095, 331.8404, 4.08295, 1.7275, 'KP'),
 

In [2]:
from astroquery.mast import Catalogs
from astropy.table import Table
from tqdm import tqdm

# Getting additional data on TESS Objects of Interest from STScI:
tic_catalog = pd.DataFrame()
for tic_id in tqdm(toi['TIC_ID'].unique()):
    row_data = Catalogs.query_criteria(catalog="Tic", ID=tic_id)
    row_data = row_data.to_pandas()
    tic_catalog = tic_catalog.append(row_data)
tic_catalog = tic_catalog.reset_index(drop=True)

# Renaming ID column to make this consistent with Caltech TOI dataframe:
tic_catalog = tic_catalog.rename(columns={'ID': 'TIC_ID'})

# Isolating columns we want:
tic_catalog = tic_catalog[['TIC_ID',
                           'ra',
                           'dec',
                           'pmRA',
                           'pmDEC',
                           'plx',
                           'gallong',
                           'gallat',
                           'eclong',
                           'eclat',
                           'Tmag',
                           'Teff',
                           'logg',
                           'MH',
                           'rad',
                           'mass',
                           'rho',
                           'lum',
                           'd',
                           'ebv',
                           'numcont',
                           'contratio',
                           'priority']]

tic_catalog.columns = tic_catalog.columns.str.replace(' ', '_')

tic_catalog.to_sql('tic_catalog', con=engine)

engine.execute("SELECT * FROM tic_catalog").fetchall()

  0%|          | 0/1133 [00:00<?, ?it/s]/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: TableReplaceWarning: converted column 'HIP' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: TableReplaceWarning: converted column 'KIC' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
  1%|          | 7/1133 [00:03<08:59,  2.09it/s]/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: TableReplaceWarning: converted column 'numcont' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 1133/1133 [06:46<00:00,  2.25it/s]


[(0, '231663901', 318.736915161395, -55.8717944482812, 12.6409, -16.0111, 2.63604, 340.697786476389, -42.0737909015092, 302.236289522646, -37.7554028753823, 12.4069, 5824.0, 4.55972, None, 0.890774, 1.05, 1.48555, 0.8224618, 375.31, 0.0333598, 90.0, 0.0844493359, 0.0005656118),
 (1, '149603524', 87.1399740872908, -63.988440787086, -15.6405, 26.0463, 5.66486, 273.415812449388, -31.0335697980936, 63.7531796047894, -87.1638479174296, 9.7109, 6391.0, 4.37665, 0.24, 1.21427, 1.28, 0.714926, 2.21616721, 175.631, 0.0202575, 971.0, 0.0163922813, 0.00709071476),
 (2, '336732616', 312.457438595966, -24.4287610609148, 10.4255, 15.6199, 2.40421, 21.2968606974153, -36.0718820310395, 308.207997713277, -6.44425872753474, 11.5232, 6371.0, 4.33006, None, 1.27616, 1.27, 0.611063, 2.41733646, 411.211, 0.0446394, 112.0, 0.02309434, 0.000414968876),
 (3, '231670397', 319.949610769276, -58.1488869526779, 10.5524, -10.6584, 3.12923, 337.503135077981, -42.1103254948997, 301.882378327253, -40.1101654151515, 9.

In [3]:
from astroquery.mast import Observations

# Getting all dataproducts for TESS Objects of Interest from STScI:
dataproducts = pd.DataFrame()
for tic_id in tqdm(toi['TIC_ID'].unique()):
    row_data = Observations.query_criteria(obs_collection="TESS",
                                           target_name=tic_id)
    row_data = row_data.to_pandas()
    dataproducts = dataproducts.append(row_data)
dataproducts = dataproducts.reset_index(drop=True)

# Renaming ID column to make this consistent with Caltech TOI dataframe:
dataproducts = dataproducts.rename(columns={'target_name': 'TIC_ID'})

# Isolating TIC IDs (target_name) and dataURL values to get associated files:
dataproducts = dataproducts[['TIC_ID', 'dataURL']]

dataproducts.columns = dataproducts.columns.str.replace(' ', '_')

dataproducts.to_sql('dataproducts', con=engine)

engine.execute("SELECT * FROM dataproducts").fetchall()

100%|██████████| 1133/1133 [02:35<00:00,  7.38it/s]


[(0, '231663901', 'mast:TESS/product/tess2018206045859-s0001-0000000231663901-0120-s_lc.fits'),
 (1, '149603524', 'mast:TESS/product/tess2019006130736-s0007-0000000149603524-0131-s_lc.fits'),
 (2, '149603524', 'mast:TESS/product/tess2018292075959-s0004-0000000149603524-0124-s_lc.fits'),
 (3, '149603524', 'mast:TESS/product/tess2019032160000-s0008-0000000149603524-0136-s_lc.fits'),
 (4, '149603524', 'mast:TESS/product/tess2018263035959-s0003-0000000149603524-0123-s_lc.fits'),
 (5, '149603524', 'mast:TESS/product/tess2018349182459-s0006-0000000149603524-0126-s_lc.fits'),
 (6, '149603524', 'mast:TESS/product/tess2018206190142-s0001-s0003-0000000149603524-00129_dvt.fits'),
 (7, '149603524', 'mast:TESS/product/tess2019058134432-s0009-0000000149603524-0139-s_lc.fits'),
 (8, '149603524', 'mast:TESS/product/tess2018206190142-s0001-s0009-0000000149603524-00205_dvt.fits'),
 (9, '149603524', 'mast:TESS/product/tess2018206190142-s0001-s0006-0000000149603524-00196_dvt.fits'),
 (10, '149603524', 'ma